[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://drive.google.com/file/d/1KI6nyX4TvG7-0z4TMEE3ND9lvHujAR0P/view?usp=sharing)

# Advanced Agent Observability with `FlotorchADKAgent` Traces

This notebook demonstrates how to build and monitor AI agents using the `FlotorchADKAgent` client. While the agent configuration (instructions, persona, model) is managed on the Flotorch platform, this guide specifically focuses on **Tracing and Observability**.

### Why Traces Matter:
In complex agentic workflows, understanding *why* an agent responded in a certain way is crucial. Tracing allows you to:
* **Debug LLM Invocations:** See exactly what system instructions and prompts were sent.
* **Monitor Performance:** Track latency across different stages (routing, guardrails, and provider calls).
* **Audit Costs:** View token usage and cost calculations per turn.
* **Verify Guardrails:** Confirm that input and output validations are functioning correctly.

### Prerequisites
1.  Configure your agent and API key in the [Flotorch Console](https://console.flotorch.cloud/).
2.  Ensure you have an active agent name and a workspace API key.

## 1. Environment Setup

We begin by installing the `flotorch` library with ADK support and its dependencies. This provides the OpenTelemetry instrumentation required to capture traces.

In [ ]:
# install flotorch adk package with dependencies
%pip install flotorch[adk]==3.1.4b1

In [ ]:
FLOTORCH_API_KEY = "sk_..."  # Update this with your flotorch Api key
FLOTORCH_BASE_URL = "https://gateway.flotorch.cloud"
AGENT_NAME = "your-agent-name"
APP_NAME = "observability_demo_app"
USER_ID = "user_001"

In [ ]:
from flotorch.adk.agent import FlotorchADKAgent
from flotorch.adk.sessions import FlotorchADKSession
from google.adk import Runner
from google.genai import types

print("Libraries imported successfully.")

## 2. Initializing Agent with Tracer Configuration

The key step for observability is passing the `tracer_config` to the `FlotorchADKAgent`. 

* **enabled**: Activates the collection of OpenTelemetry spans.
* **endpoint**: The Flotorch observability collector that aggregates your trace data.
* **sampling_rate**: Set to `1` to capture 100% of interactions (ideal for development/debugging).

In [ ]:
flotorch_client = FlotorchADKAgent(
    agent_name=AGENT_NAME,
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    tracer_config={
        "enabled": True,
        "endpoint": "<your_observability_endpoint>",
        "sampling_rate": 1 
    }
)

agent = flotorch_client.get_agent()
print(f"Agent '{agent.name}' initialized with tracing enabled.")

## 3. Runner and Session Management

We use the `Runner` to orchestrate the conversation and the `FlotorchADKSession` to maintain context. Every interaction through this runner will now generate a trace linked to the `session_id`.

In [ ]:
session_service = FlotorchADKSession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL
)
    
runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service
)

## 4. Executing an Instrumented Interaction

When we run the `chat_with_agent` function, the ADK automatically wraps the request in a trace. It captures the internal steps like session lookup, LLM routing, and the final response generation.

In [ ]:
def run_single_turn(query, session_id, user_id, runner):
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    for event in events:
        if event.is_final_response():
            if event.content and event.content.parts:
                return event.content.parts[0].text
    return "Error: No response."

async def chat_with_agent(query, session_id):
    return run_single_turn(query, session_id, USER_ID, runner)

In [ ]:
session = await runner.session_service.create_session(app_name=APP_NAME, user_id=USER_ID)
response = await chat_with_agent("Hello, I am John", session.id)
print(f"Response: {response}")

## 5. Extracting Trace Identifiers

After the interaction, we can retrieve the specific `trace_id` generated for the session. This ID acts as a unique fingerprint for the entire request-response lifecycle. You can use this ID to find logs in the Flotorch UI or to query the raw trace data programmatically.

In [ ]:
trace_ids = flotorch_client.get_tracer_ids()
print(f"Captured Trace IDs: {trace_ids}")

## 6. Inspecting Raw Trace Data

The `get_traces()` method provides a deep look into the agent's internal reasoning. The trace is structured into **Spans**, which represent individual operations:

### Key Span Components in the Output:
* **`invoke_agent`**: The root span covering the total duration of the request.
* **`call_llm`**: Contains the raw prompt and the system instruction used by the agent.
* **`guardrails.input/output`**: Details whether the content was flagged or blocked.
* **`cost.calculation`**: Breaks down `input_tokens`, `output_tokens`, and the estimated cost in USD.
* **`provider.api_call`**: Shows the specific model version (e.g., `gpt-4o`) and the provider latency.

In [ ]:
import json
traces = flotorch_client.get_traces()
print(json.dumps(traces, indent=2))

## Summary

In this notebook, we moved beyond basic agent interaction to implement a fully observable system using `FlotorchADKAgent`.

### Key Takeaways:
- **Transparent Logic**: We used traces to reveal the system instructions and model configurations being pulled from the Flotorch platform.
- **Performance Monitoring**: We identified how to track latency across multiple spans, from internal routing to external LLM calls.
- **Resource Management**: The traces provided granular token usage and cost data, essential for scaling production applications.
- **Simplified Debugging**: By using `trace_id`, developers can pinpoint failures in the Flotorch console without manually digging through logs.